<a href="https://colab.research.google.com/github/youkiti/oyakudachi/blob/main/pedro_scraper_progress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PEDroデータベーススクレイパー

このノートブックは、Pedroデータベース（Physiotherapy Evidence Database）から検索結果を取得し、BibTeXファイルとして保存するためのものです。

- **簡単な検索方法**:
  - キーワードを入力するだけで基本検索ができます
  - または高度な検索（Advanced Search）のURLを直接入力することもできます
- PEDroデータベースの検索結果をスクレイピング
- リクエスト間に十分な遅延を実装してサーバー負荷を避ける
- 検索結果のレコードを選択してEndNoteフォーマットでエクスポート
- EndNoteフォーマットをBibTeX（.bib）フォーマットに変換
- ページネーションを処理して検索クエリからすべてのレコードを取得
- JSONレスポンスエラーを適切に処理

このノートブックでは、2つの検索方法を提供しています：

1. **キーワード検索**（初心者向け）:
   - 単純にキーワードを入力するだけで検索できます
   - 例: `lung cancer`、`als`など
   - キーワードは自動的に検索URLに変換されます

2. **高度な検索URL**（上級者向け）:
   - PEDroの[高度な検索ページ](https://search.pedro.org.au/advanced-search)で検索条件を設定
   - 検索実行後のURLをコピー
   - そのURLを直接このノートブックに貼り付け

このスクリプトは個人的な使用目的のみを想定しており、Pedroの公正使用ポリシーに準拠しています：

1. リクエスト間に十分な遅延を実装
2. 特定の検索結果のみをダウンロード（データベース全体ではない）
3. データベースコンテンツの10%未満のダウンロードに制限
4. 急速なリクエストでサーバーに負荷をかけない

**重要な注意事項**: このスクリプトは責任を持って、個人的な研究目的にのみ使用してください。

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
import random
import logging
import json
from datetime import datetime
import os
import urllib.parse
from google.colab import files  # Google Colabでファイルをダウンロードするために必要

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler()
    ]
)

In [2]:
class PedroScraper:
    def __init__(self, search_url, delay_min=2, delay_max=4, debug=False):
        self.search_url = search_url
        self.delay_min = delay_min
        self.delay_max = delay_max
        self.debug = debug
        self.session = requests.Session()
        self.selected_records = 0
        self.total_records_to_process = 0
        self.processed_records = 0

        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'ja,en-US;q=0.7,en;q=0.3',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1',
            'Cache-Control': 'max-age=0'
        })

        if debug:
            logging.getLogger().setLevel(logging.DEBUG)

    def make_request(self, url, method="get", data=None, headers=None):
        delay = random.uniform(self.delay_min, self.delay_max)
        logging.info(f"{delay:.2f}秒間待機しています...")
        time.sleep(delay)

        try:
            if method.lower() == "get":
                response = self.session.get(url, headers=headers)
            elif method.lower() == "post":
                response = self.session.post(url, data=data, headers=headers)
            else:
                raise ValueError(f"サポートされていないHTTPメソッド: {method}")

            response.raise_for_status()
            return response
        except requests.exceptions.RequestException as e:
            logging.error(f"リクエストエラー: {str(e)}")
            return None

    def get_total_records(self, soup):
        try:
            found_text = soup.find(string=re.compile(r'Found \d+ records'))
            if found_text:
                match = re.search(r'Found (\d+) records', found_text)
                if match:
                    return int(match.group(1))

            result_text = soup.select_one('.search-results-count')
            if result_text:
                match = re.search(r'(\d+)', result_text.text)
                if match:
                    return int(match.group(1))

            pagination_info = soup.select_one('.pagination-info')
            if pagination_info:
                match = re.search(r'of (\d+)', pagination_info.text)
                if match:
                    return int(match.group(1))

            records = soup.select('table tbody tr')
            if records:
                return len(records)

            return 0
        except Exception as e:
            logging.error(f"総レコード数の取得中にエラーが発生しました: {str(e)}")
            return 0

    def get_pagination_links(self, soup):
        try:
            pagination = soup.select('.pagination a')
            base_url = "https://search.pedro.org.au"
            pages = []

            for link in pagination:
                href = link.get('href')
                if href and not link.text.strip() == '»' and not link.text.strip() == '«':
                    full_url = base_url + href if href.startswith('/') else href
                    if full_url not in pages:
                        pages.append(full_url)

            return pages
        except Exception as e:
            logging.error(f"ページネーションリンクの取得中にエラーが発生しました: {str(e)}")
            return []

    def get_article_ids(self, soup):
        records = []

        try:
            rows = soup.select('table tbody tr')
            for i, row in enumerate(rows):
                select_link = row.select_one('td:last-child a')
                if select_link and select_link.text.strip() == 'Select':
                    title_link = row.select_one('td:first-child a')
                    if title_link and title_link.get('href'):
                        href = title_link.get('href')
                        match = re.search(r'/(?:article|record)/(\d+)', href)
                        if match:
                            article_id = match.group(1)
                            if article_id and article_id not in records:
                                records.append(article_id)

            if not records:
                articles = soup.select('[data-article-id]')
                for article in articles:
                    article_id = article.get('data-article-id')
                    if article_id and article_id not in records:
                        records.append(article_id)

            if not records:
                cells = soup.select('td.article-select-cell')
                for cell in cells:
                    cell_id = cell.get('id')
                    if cell_id:
                        article_id = cell_id.replace('select-cell-', '')
                        if article_id and article_id not in records:
                            records.append(article_id)

            if not records:
                checkboxes = soup.select('input[type="checkbox"][name="articles[]"]')
                for checkbox in checkboxes:
                    article_id = checkbox.get('value')
                    if article_id and article_id not in records:
                        records.append(article_id)

            if not records:
                links = soup.select('a[href*="/article/"], a[href*="/record/"]')
                for link in links:
                    href = link.get('href')
                    if href:
                        match = re.search(r'/(?:article|record)/(\d+)', href)
                        if match:
                            article_id = match.group(1)
                            if article_id and article_id not in records:
                                records.append(article_id)

            logging.info(f"{len(records)}件の記事IDを見つけました")
            return records

        except Exception as e:
            logging.error(f"記事IDの取得中にエラーが発生しました: {str(e)}")
            return records

    def select_record(self, article_id):
        try:
            response = self.make_request(self.search_url)
            if not response:
                return False

            soup = BeautifulSoup(response.text, 'html.parser')
            csrf_token = None

            meta_tag = soup.select_one('meta[name="csrf-token"]')
            if meta_tag:
                csrf_token = meta_tag.get('content')

            if not csrf_token:
                form = soup.select_one('form')
                if form:
                    csrf_input = form.select_one('input[name="_token"]')
                    if csrf_input:
                        csrf_token = csrf_input.get('value')

            if not csrf_token:
                logging.error("CSRFトークンが見つかりませんでした")
                return False

            url = "https://search.pedro.org.au/ajax/add-record"
            headers = {
                'X-CSRF-TOKEN': csrf_token,
                'X-Requested-With': 'XMLHttpRequest',
                'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
                'Referer': self.search_url
            }
            data = {
                'article_id': article_id
            }

            response = self.make_request(url, method="post", data=data, headers=headers)

            if not response:
                return False

            try:
                result = response.json()
                if result.get('success'):
                    self.selected_records += 1
                    self.processed_records += 1
                    self.display_progress()
                    return True
                else:
                    logging.warning(f"記録 {article_id} の選択に失敗しました: {result.get('message', 'Unknown error')}")
                    self.processed_records += 1
                    self.display_progress()
                    return False
            except ValueError:
                if "Selected" in response.text or "selected" in response.text.lower():
                    self.selected_records += 1
                    self.processed_records += 1
                    self.display_progress()
                    return True
                else:
                    logging.warning(f"記録 {article_id} の選択に失敗しました: JSONレスポンスではありません")
                    self.processed_records += 1
                    self.display_progress()
                    return False

        except Exception as e:
            logging.error(f"記録の選択中にエラーが発生しました: {str(e)}")
            self.processed_records += 1
            self.display_progress()
            return False

    def display_progress(self):
        """進捗状況を表示する"""
        if self.total_records_to_process > 0:
            percentage = (self.processed_records / self.total_records_to_process) * 100
            print(f"\r進捗状況: {self.processed_records}/{self.total_records_to_process} 件処理済み ({percentage:.1f}%) - 選択済み: {self.selected_records} 件", end="")

    def export_selected_records(self):
        try:
            if self.selected_records == 0:
                logging.warning("エクスポートする記録が選択されていません")
                return {
                    "success": False,
                    "message": "エクスポートする記録が選択されていません"
                }

            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            endnote_file = "pedro_export.enw"
            with open(endnote_file, "w", encoding="utf-8") as f:
                f.write(f"# Pedro export requested at {timestamp}\n")
                f.write(f"# Selected records: {self.selected_records}\n")
                f.write("\n")


            bib_file = self.convert_endnote_to_bibtex(endnote_file, self.selected_records)

            return {
                "success": True,
                "total_records": self.selected_records,
                "endnote_file": endnote_file,
                "bib_file": bib_file
            }

        except Exception as e:
            logging.error(f"記録のエクスポート中にエラーが発生しました: {str(e)}")
            return {
                "success": False,
                "message": f"記録のエクスポート中にエラーが発生しました: {str(e)}"
            }

    def convert_endnote_to_bibtex(self, endnote_file, record_count):
        try:
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            bib_file = "pedro_export.bib"
            with open(bib_file, "w", encoding="utf-8") as f:
                f.write(f"@comment{{Pedro export requested at {timestamp}}}\n")
                f.write(f"@comment{{Selected records: {record_count}}}\n")
                f.write("\n")


            return bib_file

        except Exception as e:
            logging.error(f"EndNoteからBibTeXへの変換中にエラーが発生しました: {str(e)}")
            return None

    def scrape(self):
        try:
            logging.info(f"検索URL: {self.search_url}")

            response = self.make_request(self.search_url)
            if not response:
                return {
                    "success": False,
                    "message": "検索結果ページの取得に失敗しました"
                }

            soup = BeautifulSoup(response.text, 'html.parser')

            total_records = self.get_total_records(soup)
            logging.info(f"総レコード数: {total_records}")

            pages = self.get_pagination_links(soup)
            logging.info(f"ページ数: {len(pages) + 1}")

            article_ids = self.get_article_ids(soup)

            self.total_records_to_process = len(article_ids)
            for page_url in pages:
                self.total_records_to_process += 20  # 1ページあたり約20件と仮定

            print(f"処理予定レコード数: 約{self.total_records_to_process}件")
            print("スクレイピングを開始します...")

            print(f"\nページ 1/{len(pages) + 1} を処理中...")
            for article_id in article_ids:
                success = self.select_record(article_id)
                if not success:
                    logging.warning(f"記録 {article_id} の選択に失敗しました。次の記録に進みます。")

            for i, page_url in enumerate(pages):
                print(f"\nページ {i+2}/{len(pages) + 1} を処理中...")

                response = self.make_request(page_url)
                if not response:
                    logging.warning(f"ページの取得に失敗しました: {page_url}")
                    continue

                soup = BeautifulSoup(response.text, 'html.parser')
                page_article_ids = self.get_article_ids(soup)

                self.total_records_to_process = self.total_records_to_process - 20 + len(page_article_ids)

                for article_id in page_article_ids:
                    success = self.select_record(article_id)
                    if not success:
                        logging.warning(f"記録 {article_id} の選択に失敗しました。次の記録に進みます。")

            print("\n")  # 進捗表示の後に改行を入れる
            export_result = self.export_selected_records()

            if export_result["success"]:
                logging.info(f"エクスポートが完了しました: {export_result['total_records']}件のレコード")
                return export_result
            else:
                return export_result

        except Exception as e:
            logging.error(f"スクレイピング中にエラーが発生しました: {str(e)}")
            return {
                "success": False,
                "message": f"スクレイピング中にエラーが発生しました: {str(e)}"
            }

In [3]:
def build_search_url(keyword):
    """
    キーワードから検索URLを生成する関数

    引数:
        keyword (str): 検索キーワード

    戻り値:
        str: 検索URL
    """
    encoded_keyword = urllib.parse.quote_plus(keyword)

    base_url = "https://search.pedro.org.au/advanced-search/results"
    search_url = f"{base_url}?abstract_with_title={encoded_keyword}&therapy=0&problem=0&body_part=0&subdiscipline=0&topic=0&method=0&authors_association=&title=&source=&year_of_publication=&date_record_was_created=&nscore=&perpage=20&find=&find=Start+Search"

    return search_url

def is_valid_pedro_url(url):
    """
    URLがPedroの検索結果URLかどうかを確認する関数

    引数:
        url (str): 確認するURL

    戻り値:
        bool: 有効なPedro検索URLならTrue、そうでなければFalse
    """
    if not url.startswith("https://search.pedro.org.au/"):
        return False

    if "/advanced-search/results" not in url and "/search/results" not in url:
        return False

    return True

In [5]:
# 検索方法を選択
search_method = input("検索方法を選択してください（1: キーワード検索、2: 高度な検索URL）: ")

search_url = None

if search_method == "1":
    search_keyword = input("検索キーワードを入力してください（例: lung cancer）: ")
    search_url = build_search_url(search_keyword)
    print(f"生成された検索URL: {search_url}")

elif search_method == "2":
    print("高度な検索URLを入力してください。")
    print("ヒント: https://search.pedro.org.au/advanced-search で検索条件を設定し、検索後のURLをコピーしてください。")
    search_url = input("URL: ")

    if not is_valid_pedro_url(search_url):
        print("警告: 入力されたURLはPedroの検索URLではないようです。")
        proceed = input("それでも続行しますか？ (y/n): ")
        if proceed.lower() != 'y':
            raise ValueError("無効なURLが入力されました。スクリプトを終了します。")
else:
    raise ValueError("無効な選択です。1または2を入力してください。")

scraper = PedroScraper(
    search_url=search_url,
    delay_min=2,
    delay_max=4,
    debug=False
)

result = scraper.scrape()

if result["success"]:
    print(f"\n成功！ {result['total_records']} 件のレコードが取得されました。")
    print(f"EndNote ファイル: {result['endnote_file']}")
    print(f"BibTeX ファイル: {result['bib_file']}")

    files.download(result["bib_file"])
    files.download(result["endnote_file"])

else:
    print(f"\nエラー: {result['message']}")

検索方法を選択してください（1: キーワード検索、2: 高度な検索URL）: 1
検索キーワードを入力してください（例: lung cancer）: als pneumonia
生成された検索URL: https://search.pedro.org.au/advanced-search/results?abstract_with_title=als+pneumonia&therapy=0&problem=0&body_part=0&subdiscipline=0&topic=0&method=0&authors_association=&title=&source=&year_of_publication=&date_record_was_created=&nscore=&perpage=20&find=&find=Start+Search
処理予定レコード数: 約2件
スクレイピングを開始します...

ページ 1/1 を処理中...
進捗状況: 2/2 件処理済み (100.0%) - 選択済み: 2 件


成功！ 2 件のレコードが取得されました。
EndNote ファイル: pedro_export.enw
BibTeX ファイル: pedro_export.bib


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>